In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `~/research/xaqlab/ergm.jl`


In [3]:
using ergm.platinum

G = load_graph()

┌ Info: Precompiling ergm [caa93091-3dc3-4b2b-9430-1aae38e8753f]
└ @ Base loading.jl:1423


PyObject <networkx.classes.multidigraph.MultiDiGraph object at 0x7fa4b42d5090>

In [105]:
Hs_nx = subsample(G, 30, 10000)
Hs = map(nx_to_digraph, Hs_nx);

In [74]:
using ergm.stats
using ergm.spaces



In [102]:
n = 30
stats = DeltaStats(
    function(G)
        [
            sum(G.adjacency) / (n * (n - 1)),
            tr(G.adjacency ^ 3) / (n * (n - 1) * (n - 2))
        ]
    end,
    function(G, s, u)
        i, x = u
        s + [
            (x - G[i]) / (n * (n - 1)),
            3 * (x - G[i]) * dot(G.adjacency[i[2], :], G.adjacency[:, i[1]]) / (n * (n - 1) * (n - 2))
        ]
    end
)

DeltaStats(var"#37#39"(), var"#38#40"(), UndefInitializer(), UndefInitializer())

In [109]:
ms = mean(ss, dims=2)

2×1 Matrix{Float64}:
 0.010818275862069314
 1.0061576354679766e-5

0.018447568559492408

In [110]:
ss = hcat(map(H -> get_stats(stats, H), Hs)...)
d = (ss[2, :] - ss[1, :].^3) ./ (ss[1, :].^3)

10000-element Vector{Float64}:
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
   ⋮
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
  -1.0
 NaN
  -1.0

In [62]:
Hs_1 = Hs[1:50]
Hs_2 = Hs[51:100]
as = vcat(map(H -> vec(H.adjacency), Hs_1)...)
bs = vcat(map(H -> vec(H.adjacency), Hs_2)...)

500000-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [68]:
x = [sum((as .== a) .& (bs .== b)) for a ∈ 0:1, b ∈ 0:1]

2×2 Matrix{Int64}:
 492965  3298
   3698    39

In [70]:
using HypothesisTests

ChisqTest(x)

Pearson's Chi-square Test
-------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.985902, 0.00742412, 0.00662412, 4.98815e-5]
    point estimate:          [0.98593, 0.007396, 0.006596, 7.8e-5]
    95% confidence interval: [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           0.0046

Details:
    Sample size:        500000
    statistic:          8.038630248163368
    degrees of freedom: 1
    residuals:          [0.0200245, -0.230757, -0.244294, 2.81519]
    std. residuals:     [2.83525, -2.83525, -2.83525, 2.83525]
